## Advanced ML: Домашнее задание 2

Второе задание — это полноценный проект по анализу данных, начиная от анализа постановки задачи и заканчивая сравнением результатов разных моделей. Задача реальная и серьёзная, хотя тему я выбрал развлекательную: мы будем строить вероятностную рейтинг-систему для спортивного “Что? Где? Когда?” (ЧГК).  
Background: в спортивном “Что? Где? Когда?” соревнующиеся команды отвечают на одни и те же вопросы. После минуты обсуждения команды записывают и сдают свои ответы на карточках; побеждает тот, кто ответил на большее число вопросов. Турнир обычно состоит из нескольких десятков вопросов (обычно 36 или 45, иногда 60, больше редко). Часто бывают синхронные турниры, когда на одни и те же вопросы отвечают команды на сотнях игровых площадок по всему миру, т.е. в одном турнире могут играть сотни, а то и тысячи команд. Соответственно, нам нужно:   
- построить рейтинг-лист, который способен нетривиально предсказывать результаты будущих турниров;
- при этом, поскольку ЧГК — это хобби, и контрактов тут никаких нет, игроки постоянно переходят из команды в команду, сильный игрок может на один турнир сесть поиграть за другую команду и т.д.; поэтому единицей рейтинг-листа должна быть не команда, а отдельный игрок;
- а что сильно упрощает задачу и переводит её в область домашних заданий на EM-алгоритм — это характер данных: начиная с какого-то момента, в базу результатов начали вносить все повопросные результаты команд, т.е. в данных будут записи вида “какая команда на какой вопрос правильно ответила”.

Я сделал за вас только первый шаг: выкачал через API сайта рейтинга ЧГК все нужные данные, чтобы сайт не прилёг под вашими многочисленными скрейперами. :) Полученные данные лежат в формате pickle вот здесь:
https://www.dropbox.com/s/s4qj0fpsn378m2i/chgk.zip 

In [1]:
from tqdm import tqdm

import pandas as pd
import numpy as np

from scipy import sparse as sp
from scipy.special import logit, expit
from scipy.stats import kendalltau, spearmanr
from sklearn.preprocessing import OneHotEncoder

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 1.	Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
- взять в тренировочный набор турниры с dateStart из 2019 года; 
- в тестовый — турниры с dateStart из 2020 года.  

In [2]:
tournaments = pd.DataFrame(pd.read_pickle("data/tournaments.pkl").values())
players = pd.DataFrame(pd.read_pickle("data/players.pkl").values())

results = pd.read_pickle("data/results.pkl")

**Подготовим данные:**

In [3]:
players.head()

,id,name,patronymic,surname
0,1,Алексей,None,Абабилов
1,10,Игорь,,Абалов
2,11,Наталья,Юрьевна,Абалымова
3,12,Артур,Евгеньевич,Абальян
4,13,Эрик,Евгеньевич,Абальян


In [4]:
players["name"] = players["name"] + " " + players["surname"]
players = players[["id", "name"]]
players.head()

,id,name
0,1,Алексей Абабилов
1,10,Игорь Абалов
2,11,Наталья Абалымова
3,12,Артур Абальян
4,13,Эрик Абальян


In [5]:
any(players["id"].duplicated())

False

In [6]:
tournaments.head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
0,1,Чемпионат Южного Кавказа,2003-07-25T00:00:00+04:00,2003-07-27T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
1,2,Летние зори,2003-08-09T00:00:00+04:00,2003-08-09T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
2,3,Турнир в Ижевске,2003-11-22T00:00:00+03:00,2003-11-24T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
3,4,Чемпионат Украины. Переходной этап,2003-10-11T00:00:00+04:00,2003-10-12T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
4,5,Бостонское чаепитие,2003-10-10T00:00:00+04:00,2003-10-13T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None


In [7]:
tournaments["year"] = pd.DatetimeIndex(pd.to_datetime(tournaments["dateStart"], utc=True)).year
tournaments_train = tournaments.loc[tournaments["year"]==2019, ["id", "name"]]
tournaments_test = tournaments.loc[tournaments["year"]==2020, ["id", "name"]]
tournaments_train.head()

,id,name
3921,4772,Синхрон северных стран. Зимний выпуск
4115,4973,Балтийский Берег. 3 игра
4116,4974,Балтийский Берег. 4 игра
4117,4975,Балтийский Берег. 5 игра
4128,4986,ОВСЧ. 6 этап


In [8]:
any(tournaments["id"].duplicated())

False

**Сформируем датафрейм:**

In [9]:
def make_df_from_results(results, tournaments_df):
    results_df = {
        "tournament_id": [],
        "team_id": [],
        "player_id": [],
        "question_id": [],
        "answer": [],
        "n_right_answers": [],
        "n_questions": []
    }
    teams_df = {
        "id": [],
        "name": [],
        "position": [],
        "town": []
    }
    for tournament_id, teams_data in tqdm(results.items()):
        if tournament_id not in tournaments_df["id"].values:
            continue
        tournament_result = {}
        mask_len = None
        for team in teams_data:
            mask = team.get("mask")
            players = [
                player["player"]["id"] for player in team["teamMembers"]
            ]

            # Не будем рассматривать команды в которых нет игрооков или макски отвтетов
            if mask is None or not players:
                continue
            # Так же будем следить, чтобы у каждой команды на турнире, было одинаковое кол-во вопросов
            if mask_len is None:
                mask_len = len(mask)
            elif len(mask) != mask_len:
                tournament_result = {}
                break

            team_id = team["team"]["id"]
            tournament_result[team_id] = {}
            tournament_result[team_id]["team_name"] = team["team"]["name"]
            tournament_result[team_id]["mask"] = mask
            tournament_result[team_id]["players"] = players
            tournament_result[team_id]["team_position"] = team["position"]
            tournament_result[team_id]["team_town"] = team["team"].get("town").get("name") if team["team"].get("town") is not None else None
        
        if tournament_result:
            for team_id, team_data in tournament_result.items():
                temp_questions = []
                temp_answers = []
                temp_players = []
                for i, answer in enumerate(team_data["mask"]):
                    temp_questions.append(str(tournament_id) + "_" + str(i))
                    answer = (
                        0 if answer == "X" or answer == "?" else int(answer)
                    )
                    temp_answers.append(answer)
                n_questions = len(temp_answers)
                n_players = len(team_data["players"])
                for player_id in team_data["players"]:
                    temp_players += [player_id] * n_questions

                results_df["tournament_id"] += (
                    [tournament_id] * n_players * n_questions
                )
                results_df["team_id"] += [team_id] * n_players * n_questions
                results_df["player_id"] += temp_players
                results_df["question_id"] += temp_questions * n_players
                results_df["answer"] += temp_answers * n_players
                results_df["n_right_answers"] += [sum(temp_answers)] * n_players * n_questions
                results_df["n_questions"] += [len(temp_answers)] * n_players * n_questions
                
                teams_df["id"].append(team_id)
                teams_df["name"].append(team_data["team_name"])
                teams_df["position"].append(team_data["team_position"])
                teams_df["town"].append(team_data["team_town"])

    results_df = pd.DataFrame(results_df)
    teams_df = pd.DataFrame(teams_df)
    return results_df, teams_df


In [10]:
train_df, teams_train = make_df_from_results(results, tournaments_train)
test_df, teams_test = make_df_from_results(results, tournaments_test)

100%|██████████| 5528/5528 [00:01<00:00, 3361.83it/s] 


In [11]:
train_df.head()

,tournament_id,team_id,player_id,question_id,answer,n_right_answers,n_questions
0,4772,45556,6212,4772_0,1,28,36
1,4772,45556,6212,4772_1,1,28,36
2,4772,45556,6212,4772_2,1,28,36
3,4772,45556,6212,4772_3,1,28,36
4,4772,45556,6212,4772_4,1,28,36


### 2.	Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


Используем One Hot Encoder для получения отдельных коэффициентов для каждого игрока и вопроса.
Обучим LogisticRegression на данных x = [игроки, вопросы], y = [правильный ответ или нет].    
Коэффициент при player_id можно интерпретировать как "силу" игрока.  
Коэффициент при question_id можно интерпретировать на сколько он был "лёгкий".

In [38]:
enc = OneHotEncoder(handle_unknown="ignore", dtype=np.int8)
X_train = enc.fit_transform(train_df[["player_id", "question_id"]])
y_train = np.array(train_df["answer"], dtype=np.int32)


train_players = np.unique(train_df["player_id"])
test_df = test_df.loc[test_df["player_id"].isin(train_players), :]
# Заменим question_id, иммитируя, что мы не имеем никакой информации о вопросе
test_df["unknown_question_id"] = "-1"

X_test = enc.transform(test_df[["player_id", "unknown_question_id"]])
y_test = np.array(train_df["answer"], dtype=np.int32)

In [39]:
X_train = X_train.tocoo()

X_train = torch.sparse.FloatTensor(
    torch.LongTensor(np.vstack((X_train.row, X_train.col))),
    torch.FloatTensor(X_train.data),
    torch.Size(X_train.shape),
).to(device)

y_train = torch.FloatTensor(y_train).view(-1, 1).to(device)


X_test = X_test.tocoo()

X_test = torch.sparse.FloatTensor(
    torch.LongTensor(np.vstack((X_test.row, X_test.col))),
    torch.FloatTensor(X_test.data),
    torch.Size(X_test.shape),
).to(device)

y_test = torch.FloatTensor(y_test).view(-1, 1).to(device)


In [12]:
class LogisticRegression(nn.Module):
    def __init__(self, n_features):  
        super().__init__()
        self.linear = nn.Linear(n_features, 1)
        self.sigmoid = nn.Sigmoid()     
    
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [15]:
input_dim = X_train.shape[1]
output_dim = 1 
learning_rate = 1

model = LogisticRegression(input_dim).to(device)
criterion = torch.nn.BCELoss(size_average=True)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def train_model(x, y, model, criterion, optimizer, verbose_step=50):
    for epoch in range(epochs):
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        if epoch % verbose_step == 0:
            loss = loss.item()
            print(f"loss: {loss:>7f}  [{epoch:>5d}/{epochs:>5d}]")

In [16]:
epochs = 200
train_model(X_train, y_train, model, criterion, optimizer)

loss: 0.693064  [    0/  200]
loss: 0.461102  [   50/  200]
loss: 0.460019  [  100/  200]
loss: 0.460013  [  150/  200]


In [18]:
players_rating = pd.DataFrame(
    {
        "id": train_players,
        "lr_rating": model.linear.weight.data[0].cpu().numpy()[: train_players.shape[0]],
    }
)

players_rating = players_rating.join(players.set_index("id"), on="id").sort_values(by="lr_rating", ascending=False)
players_rating.head(20)

,id,lr_rating,name
3859,27403,3.261256,Максим Руссо
604,4270,3.125218,Александра Брутер
4049,28751,3.073102,Иван Семушин
3932,27822,2.985342,Михаил Савченков
5554,40411,2.914508,Дмитрий Кудинов
4257,30270,2.911552,Сергей Спешков
4237,30152,2.900370,Артём Сорожкин
5355,38175,2.840075,Максим Пилипенко
2925,20691,2.802929,Станислав Мереминский
2567,18036,2.749244,Михаил Левандовский


Выгрузил топ-20 игроков на 02.01.2020 с [сайта](https://rating.chgk.info/). Получившийся рейтинг согласуется с тем, что предствален на сайте:

In [19]:
top_20 = pd.read_excel("data/top20_02.01.2020.xls")
top_20["id"] = top_20["id"].astype(int)
pd.merge(players_rating.head(20), top_20, how='inner', on=["id"])

,id,lr_rating,name_x,name_y
0,27403,3.261256,Максим Руссо,Руссо Максим Михайлович
1,4270,3.125218,Александра Брутер,Брутер Александра Владимировна
2,28751,3.073102,Иван Семушин,Семушин Иван Николаевич
3,27822,2.985342,Михаил Савченков,Савченков Михаил Владимирович
4,30270,2.911552,Сергей Спешков,Спешков Сергей Леонидович
5,30152,2.900370,Артём Сорожкин,Сорожкин Артём Сергеевич
6,22799,2.724769,Сергей Николенко,Николенко Сергей Игоревич
7,18332,2.623016,Александр Либер,Либер Александр Витальевич


### 3.	Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.  
*(hint: Для самопроверки: у меня средняя корреляция Спирмена на тестовом множестве 2020 года во всех моделях, включая baselines, получалась порядка 0.7-0.8, а корреляция Кендалла — порядка 0.5-0.6. Если у вас корреляции вышли за 0.9 или, наоборот, упали ниже 0.3, скорее всего где-то баг.)*

In [20]:
test_df.head()

,tournament_id,team_id,player_id,question_id,answer,n_right_answers,n_questions,unknown_question_id
0,4957,49804,30152,4957_0,1,26,39,-1
1,4957,49804,30152,4957_1,1,26,39,-1
2,4957,49804,30152,4957_2,1,26,39,-1
3,4957,49804,30152,4957_3,1,26,39,-1
4,4957,49804,30152,4957_4,1,26,39,-1


Для команд за "вероятность" ответа на вопрос, возьмём вероятность, что хотя бы один участник этой команды правильно ответил на вопрос: 
$$p(team=1) = 1 - \prod\big(1-p(player=1)\big)$$

In [21]:
with torch.no_grad():
    players_prob = model(X_test).flatten().cpu().numpy()

test_df["player_answer_prob"] = players_prob
team_ratings = test_df[["tournament_id", "team_id", "player_id", "n_right_answers", "player_answer_prob"]].drop_duplicates()

team_ratings["team_answer_prob"] = team_ratings.groupby(["tournament_id", "team_id"])["player_answer_prob"].transform(lambda x: 1 - np.prod(1 - x))

team_ratings = team_ratings.sort_values(by=["tournament_id", "n_right_answers"], ascending=False) 
team_ratings["true_rating"] = team_ratings.groupby("tournament_id")["n_right_answers"].transform(lambda x: np.arange(1, len(x) + 1))


team_ratings = team_ratings.sort_values(by=["tournament_id", "team_answer_prob"], ascending=False) 
team_ratings["pred_rating"] = team_ratings.groupby("tournament_id")["team_answer_prob"].transform(lambda x: np.arange(1, len(x) + 1))
team_ratings["pred_rating"] = team_ratings["pred_rating"].astype(np.int32)

In [22]:
print(
    f"Корреляция Спирмена: {team_ratings.groupby('tournament_id').apply(lambda x: spearmanr(x['true_rating'], x['pred_rating']).correlation).mean()}"
)
print(
    f"Корреляция Кендалла: {team_ratings.groupby('tournament_id').apply(lambda x: kendalltau(x['true_rating'], x['pred_rating']).correlation).mean()}"
)


Корреляция Спирмена: 0.8076626473393483
Корреляция Кендалла: 0.649420794720778


### 4.	Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
- предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
- разработайте EM-схему для обучения этой модели, реализуйте её в коде;
- обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


Как было отмечено выше, ЧГК - командная игра. А с теми упрощениями, что мы делали выше, получается, что каждый игрок отвечает на каждый вопрос. Понятно, что это не так, но проблема в том, что мы не знаем какой конкртено ирок ответил на данный команде вопрос. Чтобы это учесть для каждой пары игрок-вопрос, добвавим переменную, которая будет показывать, что именно этот игрок ответил на этот вопрос.
В итоге получим:  
* на Е-шаге оцениваем переменную $p(player=1|team=1)$: $$p(player=1|team) = answer * \frac{p(player=1)}{p(team=1)},$$
где answer - показывает, правильно ли ответила команда на вопрос, если команда ответила не правильно (answer=0), то ни один участник команды не ответил правильно на вопрос.
* на M-шаге, используем эту переменную как таргет для обучения LogisticRegression. После этого шага, мы получаем:
    1. "уточнённые" вероятности для игроков, используем их на очередном E-шаге.
    2. "уточнённые" коэффициенты, т.е. обновлённый рейтинг игроков (и вопросов). 

In [13]:
class ExpectationMaximizationLogisticRegression:
    def __init__(self, n_steps=5, epochs=100, learning_rate=1, verbose=None):
        self.n_steps = n_steps
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.verbose = verbose
        self.model = None
        self.criterion = None
        self.optimizer = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def e_step(self, player_probs, train_data):
        temp_df = train_data.copy()
        temp_df["team_answer_prob"] = player_probs
        temp_df.loc[temp_df["answer"] == 0, "team_answer_prob"] = 0
        idx = temp_df["answer"] == 1
        sp = temp_df.loc[idx].groupby(["team_id", "question_id"])["team_answer_prob"].transform(lambda x: 1 - np.prod(1 - x.values))
        temp_df.loc[idx, "team_answer_prob"] = temp_df.loc[idx, "team_answer_prob"] / sp  
        return temp_df["team_answer_prob"].values


    def m_step(self, X, y):
        y = torch.FloatTensor(y).view(-1, 1).to(self.device)
        self.model.linear.reset_parameters()
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        for epoch in range(self.epochs):
            self.optimizer.zero_grad()
            output = self.model(X)
            loss = self.criterion(output, y)
            loss.backward()
            self.optimizer.step()
        y = self.predict_proba(X)
        return y        
    
    def fit(self, X_train, train_data, X_test=None, test_data=None):
        """
        X - OneHotEncoding of player_id, question_id
        data has [team_id, player_id, question_id, answer] columns"""
        if self.model is None:
            input_dim = X_train.shape[1]
            self.model = LogisticRegression(input_dim).to(self.device)
            self.criterion = nn.BCELoss()
            self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        
        if test_data is not None:
            assert (
                test_data["player_id"].isin(train_data["player_id"].unique()).sum()
                == test_data.shape[0] 
            ), "some players in test don't present in train!"

        if self.verbose is None and X_test is not None:
            self.verbose = 2

        X_train = self._prep_data(X_train)
        player_probs = self.m_step(X_train, train_df["answer"].values)
        
        if X_test is not None:
            X_test = self._prep_data(X_test)
            print(f"{'=' * 15} start correlations: {'=' * 15}")
            spearman_cor, kendall_cor = self.get_correlations_score(X_test, test_data)
            print(f"Корреляция Спирмена: {spearman_cor}")
            print(f"Корреляция Кендалла: {kendall_cor}")
        for step in range(1, self.n_steps + 1):
            print(f"{'=' * 15} step={step}/{self.n_steps} {'=' * 15}")
            player_probs = self.e_step(player_probs, train_data)
            player_probs = self.m_step(X_train, player_probs)
            if self.verbose is not None and step % self.verbose == 0:
                spearman_cor, kendall_cor = self.get_correlations_score(X_test, test_data)
                print(f"Корреляция Спирмена: {spearman_cor}")
                print(f"Корреляция Кендалла: {kendall_cor}")

    def _prep_data(self, X):
        X = X.tocoo()
        X = torch.sparse.FloatTensor(
            torch.LongTensor(np.vstack((X.row, X.col))),
            torch.FloatTensor(X.data),
            torch.Size(X.shape),
        ).to(self.device)
        return X

    def _predict_proba(self, X):
        with torch.no_grad():
            players_prob = self.model(X)
        return players_prob

    def predict_proba(self, X):
        if type(X) == sp.csr.csr_matrix:
            X = self._prep_data(X)
        with torch.no_grad():
            players_prob = self.model(X).cpu().detach().numpy().ravel()
        return players_prob

    def predict(self, X):
        return (self.predict_proba(X) > 0.5).astype(np.int32)

    def get_correlations_score(self, X_test, test_data):
        temp_df = test_data.copy()
        temp_df["player_answer_prob"] = self.predict_proba(X_test)  
        team_ratings = temp_df[["tournament_id", "team_id", "player_id", "n_right_answers", "player_answer_prob"]].drop_duplicates()
        team_ratings["team_answer_prob"] = team_ratings.groupby(["tournament_id", "team_id"])["player_answer_prob"].transform(lambda x: 1 - np.prod(1 - x))
        team_ratings = team_ratings.sort_values(by=["tournament_id", "n_right_answers"], ascending=False) 
        team_ratings["true_rating"] = team_ratings.groupby("tournament_id")["n_right_answers"].transform(lambda x: np.arange(1, len(x) + 1))
        team_ratings = team_ratings.sort_values(by=["tournament_id", "team_answer_prob"], ascending=False) 
        team_ratings["pred_rating"] = team_ratings.groupby("tournament_id")["team_answer_prob"].transform(lambda x: np.arange(1, len(x) + 1))
        team_ratings["pred_rating"] = team_ratings["pred_rating"].astype(np.int32)
        spearman_cor = team_ratings.groupby('tournament_id').apply(lambda x: spearmanr(x['true_rating'], x['pred_rating']).correlation).mean()
        kendall_cor = team_ratings.groupby('tournament_id').apply(lambda x: kendalltau(x['true_rating'], x['pred_rating']).correlation).mean()
        return spearman_cor, kendall_cor

In [14]:
enc = OneHotEncoder(handle_unknown="ignore", dtype=np.int8)
X_train = enc.fit_transform(train_df[["player_id", "question_id"]])

train_players = np.unique(train_df["player_id"])
test_df = test_df.loc[test_df["player_id"].isin(train_players), :]
# Заменим question_id, иммитируя, что мы не имеем никакой информации о вопросе
test_df["unknown_question_id"] = "-1"
X_test = enc.transform(test_df[["player_id", "unknown_question_id"]])

In [15]:
em_cl = ExpectationMaximizationLogisticRegression(n_steps=10, epochs=100, learning_rate=1, verbose=1)

In [16]:
em_cl.fit(X_train, train_df, X_test, test_df)

=============== start correlations: ===============
Корреляция Спирмена: 0.8073315753283674
Корреляция Кендалла: 0.6493196407954187
=============== step=1/10 ===============
Корреляция Спирмена: 0.8153948803525982
Корреляция Кендалла: 0.6576169004668548
=============== step=2/10 ===============
Корреляция Спирмена: 0.8175669325064544
Корреляция Кендалла: 0.6594368057384556
=============== step=3/10 ===============
Корреляция Спирмена: 0.8178158398655462
Корреляция Кендалла: 0.6598730554859347
=============== step=4/10 ===============
Корреляция Спирмена: 0.819619047548335
Корреляция Кендалла: 0.662452895951778
=============== step=5/10 ===============
Корреляция Спирмена: 0.8194514671141288
Корреляция Кендалла: 0.6622905010886097
=============== step=6/10 ===============
Корреляция Спирмена: 0.8187588376625651
Корреляция Кендалла: 0.6622767201297437
=============== step=7/10 ===============
Корреляция Спирмена: 0.8185139163445033
Корреляция Кендалла: 0.6619430097487851
=============== 

### 5.	А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)?
Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

In [27]:
questions = train_df["question_id"].unique()
question_complexity = dict(zip(questions, model.linear.weight.data[0].cpu().numpy()[-len(questions):]))
tournament_rating = train_df[["tournament_id", "question_id"]].drop_duplicates()
tournament_rating["question_complexity"] = tournament_rating["question_id"].map(question_complexity)
tournament_rating = tournament_rating.merge(tournaments[["id", "name"]], left_on="tournament_id", right_on="id", how="left")
tournament_rating = tournament_rating.groupby('name')['question_complexity'].mean().sort_values().reset_index()

In [28]:
tournament_rating.head(20)

,name,question_complexity
0,Чемпионат Санкт-Петербурга. Первая лига,-10.421551
1,Зеркало мемориала памяти Михаила Басса,-3.631952
2,Чемпионат Таджикистана,-3.497978
3,Синхрон высшей лиги Москвы,-3.013088
4,Воображаемый музей,-2.957678
5,День D,-2.797349
6,Мемориал памяти Михаила Басса,-2.784051
7,Угрюмый Ёрш,-2.777620
8,Шестая октава: СИ-Мажор. Лига Наций: Прибалтика,-2.730512
9,Поволжская лига,-2.693988


In [29]:
tournament_rating.tail(20)

,name,question_complexity
635,Кубок Закарпатья,2.334564
636,Второй тематический турнир имени Джоуи Триббиани,2.344079
637,Межфакультетский кубок МГУ. Отбор №4,2.366461
638,Школьный Синхрон-lite. Выпуск 2.3,2.391044
639,Шестой киевский марафон. Асинхрон,2.421593
640,Малый кубок Физтеха,2.437323
641,Школьный Синхрон-lite. Выпуск 3.3,2.449002
642,(а)Синхрон-lite. Лига старта. Эпизод IX,2.471185
643,(а)Синхрон-lite. Лига старта. Эпизод III,2.473452
644,(а)Синхрон-lite. Лига старта. Эпизод XI,2.483283


Судя по названиям, рейтинг кажется релевантным.

### 6.	Бонус: постройте топ игроков по предсказанной вашей моделью силе игры, а рядом с именами игроков напишите общее число вопросов, которое они сыграли.
Скорее всего, вы увидите, что топ занят игроками, которые сыграли совсем мало вопросов, около 100 или даже меньше; если вы поищете их в официальном рейтинге ЧГК, вы увидите, что это какие-то непонятные ноунеймы . В baseline-модели, скорее всего, такой эффект будет гораздо слабее.  
*(hint: Для самопроверки: а вот те игроки, кто сыграл от тысячи вопросов и больше и при этом всё равно попал в топ-100 весов модели, должны быть настоящими топовыми игроками из ведущих команд официального рейтинга. Если это не так, опять же, скорее всего где-то баг.)*  
Это естественное свойство модели: за счёт EM-схемы влияние 1-2 удачно сыгранных турниров будет только усиливаться, потому что неудачных турниров, чтобы его компенсировать, у этих игроков нет. Более того, это не мешает метрикам качества, потому что если эти игроки сыграли всего 1-2 турнира в 2019-м, скорее всего они ничего или очень мало сыграли и в 2020, и их рейтинги никак не влияют на качество тестовых предсказаний. Но для реального рейтинга такое свойство, конечно, было бы крайне нежелательным. Давайте попробуем его исправить:
- сначала жёстко: выберите разумную отсечку по числу вопросов, учитывая, что в одном турнире их обычно 30-50;
- можно ли просто выбросить игроков, которые мало играли, и переобучить модель? почему? предложите, как нужно изменить модель, чтобы не учитывать слишком мало сыгравших, и переобучите модель;
- но всё-таки это не слишком хорошее решение: если выбрать маленькую отсечку, будут ноунеймы в топе, а если большую, то получится, что у нового игрока слишком долго не будет рейтинга; скорее всего, никакой “золотой середины” тут не получится;
- предложите более концептуальное решение для топа игроков в рейтинг-листе; если получится, реализуйте его на практике (за это уж точно будут серьёзные бонусные баллы).  


In [17]:
em_players_rating = pd.DataFrame(
    {
        "id": train_players,
        "em_rating": em_cl.model.linear.weight.data[0].cpu().numpy()[: train_players.shape[0]],
    }
)

em_players_rating = em_players_rating.join(players.set_index("id"), on="id").sort_values(by="em_rating", ascending=False)
em_players_rating["questions_count"] = em_players_rating["id"].map(train_df.groupby("player_id")["question_id"].count())
em_players_rating.head(20)

,id,em_rating,name,questions_count
5355,38175,3.080345,Максим Пилипенко,36
3180,22474,3.056233,Илья Немец,75
2114,14996,2.596238,Ольга Козлова,36
39846,199963,2.589726,Елена Бровченко,36
42041,202410,2.502628,Валентина Подюкова,36
5554,40411,2.465606,Дмитрий Кудинов,45
4686,33459,2.370977,Феликс Фрайман,36
29722,188876,2.355033,Мария Голудина,36
39068,199112,2.315164,Игорь Петров,30
39069,199113,2.315150,Андриан Иоаннидис,30


Действительно в топе оказалось много игроков с малым количеством сыгранных турниров.  
Чтобы наша модель учитывала это - добавим регуляризацию и будем штрафовать веса за "малое" количество вопросов: $$loss += w * question\_reg\_coef * question\_reg\_treshold / questions\_count$$

In [19]:
class ExpectationMaximizationLogisticRegression:
    def __init__(self, n_steps=5, epochs=100, learning_rate=1, question_reg_coef=None, question_reg_treshold=None, verbose=None):
        self.n_steps = n_steps
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.verbose = verbose
        self.model = None
        self.criterion = None
        self.optimizer = None
        self.question_reg_coef = question_reg_coef
        self.question_reg_treshold = question_reg_treshold
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def e_step(self, player_probs, train_data):
        temp_df = train_data.copy()
        temp_df["team_answer_prob"] = player_probs
        temp_df.loc[temp_df["answer"] == 0, "team_answer_prob"] = 0
        idx = temp_df["answer"] == 1
        sp = temp_df.loc[idx].groupby(["team_id", "question_id"])["team_answer_prob"].transform(lambda x: 1 - np.prod(1 - x.values))
        temp_df.loc[idx, "team_answer_prob"] = temp_df.loc[idx, "team_answer_prob"] / sp  
        return temp_df["team_answer_prob"].values


    def m_step(self, X, y, questions_count):
        y = torch.FloatTensor(y).view(-1, 1).to(self.device)
        self.model.linear.reset_parameters()
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        for epoch in range(self.epochs):
            self.optimizer.zero_grad()
            output = self.model(X)
            # добавим дополнительную регуляризацию 
            loss = self.criterion(output, y) + np.mean(self.question_reg_coef * self.model.linear.weight.data.cpu().numpy()[0][: train_players.shape[0]] *  self.question_reg_treshold / questions_count )
            loss.backward()
            self.optimizer.step()
        y = self.predict_proba(X)
        return y        
    
    def fit(self, X_train, train_data, X_test=None, test_data=None):
        """
        X - OneHotEncoding of player_id, question_id
        data has [team_id, player_id, question_id, answer] columns"""
        if self.model is None:
            input_dim = X_train.shape[1]
            self.model = LogisticRegression(input_dim).to(self.device)
            self.criterion = nn.BCELoss()
            self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        
        questions_count = train_df.groupby("player_id")["question_id"].count().values
        if test_data is not None:
            assert (
                test_data["player_id"].isin(train_data["player_id"].unique()).sum()
                == test_data.shape[0] 
            ), "some players in test don't present in train!"

        if self.verbose is None and X_test is not None:
            self.verbose = 2

        X_train = self._prep_data(X_train)
        player_probs = self.m_step(X_train, train_df["answer"].values, questions_count)
        
        if X_test is not None:
            X_test = self._prep_data(X_test)
            print(f"{'=' * 15} start correlations: {'=' * 15}")
            spearman_cor, kendall_cor = self.get_correlations_score(X_test, test_data)
            print(f"Корреляция Спирмена: {spearman_cor}")
            print(f"Корреляция Кендалла: {kendall_cor}")
        for step in range(1, self.n_steps + 1):
            print(f"{'=' * 15} step={step}/{self.n_steps} {'=' * 15}")
            player_probs = self.e_step(player_probs, train_data)
            player_probs = self.m_step(X_train, player_probs, questions_count)
            if self.verbose is not None and step % self.verbose == 0:
                spearman_cor, kendall_cor = self.get_correlations_score(X_test, test_data)
                print(f"Корреляция Спирмена: {spearman_cor}")
                print(f"Корреляция Кендалла: {kendall_cor}")

    def _prep_data(self, X):
        X = X.tocoo()
        X = torch.sparse.FloatTensor(
            torch.LongTensor(np.vstack((X.row, X.col))),
            torch.FloatTensor(X.data),
            torch.Size(X.shape),
        ).to(self.device)
        return X

    def _predict_proba(self, X):
        with torch.no_grad():
            players_prob = self.model(X)
        return players_prob

    def predict_proba(self, X):
        if type(X) == sp.csr.csr_matrix:
            X = self._prep_data(X)
        with torch.no_grad():
            players_prob = self.model(X).cpu().detach().numpy().ravel()
        return players_prob

    def predict(self, X):
        return (self.predict_proba(X) > 0.5).astype(np.int32)

    def get_correlations_score(self, X_test, test_data):
        temp_df = test_data.copy()
        temp_df["player_answer_prob"] = self.predict_proba(X_test)  
        team_ratings = temp_df[["tournament_id", "team_id", "player_id", "n_right_answers", "player_answer_prob"]].drop_duplicates()
        team_ratings["team_answer_prob"] = team_ratings.groupby(["tournament_id", "team_id"])["player_answer_prob"].transform(lambda x: 1 - np.prod(1 - x))
        team_ratings = team_ratings.sort_values(by=["tournament_id", "n_right_answers"], ascending=False) 
        team_ratings["true_rating"] = team_ratings.groupby("tournament_id")["n_right_answers"].transform(lambda x: np.arange(1, len(x) + 1))
        team_ratings = team_ratings.sort_values(by=["tournament_id", "team_answer_prob"], ascending=False) 
        team_ratings["pred_rating"] = team_ratings.groupby("tournament_id")["team_answer_prob"].transform(lambda x: np.arange(1, len(x) + 1))
        team_ratings["pred_rating"] = team_ratings["pred_rating"].astype(np.int32)
        spearman_cor = team_ratings.groupby('tournament_id').apply(lambda x: spearmanr(x['true_rating'], x['pred_rating']).correlation).mean()
        kendall_cor = team_ratings.groupby('tournament_id').apply(lambda x: kendalltau(x['true_rating'], x['pred_rating']).correlation).mean()
        return spearman_cor, kendall_cor

In [24]:
em_cl = ExpectationMaximizationLogisticRegression(n_steps=1, epochs=100, learning_rate=1, verbose=1, question_reg_coef=0.5, question_reg_treshold=500)

In [25]:
em_cl.fit(X_train, train_df, X_test, test_df)

=============== start correlations: ===============
Корреляция Спирмена: 0.8076863118908355
Корреляция Кендалла: 0.6494824697061439
=============== step=1/1 ===============
Корреляция Спирмена: 0.8153953387056402
Корреляция Кендалла: 0.6576033407559336


In [26]:
em_players_rating = pd.DataFrame(
    {
        "id": train_players,
        "em_rating": em_cl.model.linear.weight.data[0].cpu().numpy()[: train_players.shape[0]],
    }
)

em_players_rating = em_players_rating.join(players.set_index("id"), on="id").sort_values(by="em_rating", ascending=False)
em_players_rating["questions_count"] = em_players_rating["id"].map(train_df.groupby("player_id")["question_id"].count())
em_players_rating.head(20)

,id,em_rating,name,questions_count
3859,27403,3.207182,Максим Руссо,2195
604,4270,3.026563,Александра Брутер,2711
4049,28751,2.972983,Иван Семушин,3803
3932,27822,2.886860,Михаил Савченков,3236
5554,40411,2.879085,Дмитрий Кудинов,45
5355,38175,2.841146,Максим Пилипенко,36
4237,30152,2.784841,Артём Сорожкин,4885
4257,30270,2.782692,Сергей Спешков,3767
2925,20691,2.696316,Станислав Мереминский,1595
3180,22474,2.691442,Илья Немец,75


Можно ещё поиграть с параметрами, но результат получился более адекватным!

### 7.	Бонус: игроки со временем учатся играть лучше (а иногда бывает и наоборот). 
А в нашей модели получается, что первые неудачные турниры новичка будут тянуть его рейтинг вниз всю жизнь — это нехорошо, рейтинг должен быть достаточно гибким и иметь возможность меняться даже у игроков, отыгравших сотни турниров. Давайте попробуем этого добиться:
- если хватит вычислительных ресурсов, сначала сделайте baseline совсем без таких схем, обучив рейтинги на всех турнирах с повопросными результатами, а не только на турнирах 2019 года; улучшилось ли качество предсказаний на 2020?
- одну схему со временем мы уже использовали: брали для обучения только последний год турниров; примерно так делают, например, в теннисной чемпионской гонке; у этой схемы есть свои преимущества, но есть и недостатки (например, достаточно мало играть год, чтобы полностью пропасть из рейтинга);
- предложите варианты базовой модели или алгоритма её обучения, которые могли бы реализовать изменения рейтинга со временем; если получится, реализуйте их на практике, проверьте, улучшатся ли предсказания на 2020.